In [67]:
from game import Game
import time
import os
import re

## Load maps

In [68]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [69]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [70]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [71]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [72]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [73]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [74]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [75]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


## Solvers

In [76]:
class State:
    def __init__(self, player_loc, boxes_loc, path):
        self.player_loc_ = player_loc
        self.boxes_loc_ = boxes_loc
        self.path_ = path
    
    def __hash__(self):
        return hash((self.player_loc_, tuple(self.boxes_loc_)))

    def __eq__(self, other):
        return (self.player_loc_, self.boxes_loc_) == (other.player_loc_, other.boxes_loc_)
    
    def __lt__(self, other):
        return len(self.path_) < len(other.path_)
    
    def get_boxes_loc(self):
        return self.boxes_loc_
    
    def get_player_loc(self):
        return self.player_loc_
    
    def get_path(self):
        return self.path_

In [77]:
from collections import deque
import heapq

### BFS

In [78]:
# TODO: Must return moves (if there is no solution return None), number of visited states
def solver_bfs(game_map):
    game = Game(game_map)
    initial_state = State(game.get_player_position(), game.get_box_locations(), "")
    frontier = deque([initial_state])
    explored = set()
    explored.add(initial_state)
    if game.is_game_won():
        return initial_state.get_path(), len(explored)
    
    while frontier:
        node = frontier.popleft()
        game.set_box_positions(node.get_boxes_loc())
        game.set_player_position(node.get_player_loc())
        
        for action in ["U", "D", "L", "R"]:
            if game.apply_move(action):
                new_state = State(game.get_player_position(), game.get_box_locations(), node.get_path() + action)
                if new_state not in explored:
                    frontier.append(new_state)
                    explored.add(new_state)
                    if game.is_game_won():
                        return new_state.get_path(), len(explored)
                
                game.set_box_positions(node.get_boxes_loc())
                game.set_player_position(node.get_player_loc())
                
    return None, len(explored)

### DFS

In [79]:
# TODO: Must return moves, number of visited states
def solver_dfs(game_map):
    game = Game(game_map)
    initial_state = State(game.get_player_position(), game.get_box_locations(), "")
    frontier = [initial_state]
    explored = set()
    
    while frontier:
        node = frontier.pop()
        explored.add(node)
        game.set_box_positions(node.get_boxes_loc())
        game.set_player_position(node.get_player_loc())
        if game.is_game_won():
            return node.get_path(), len(explored)
        
        for action in ["U", "D", "L", "R"]:
            if game.apply_move(action):
                new_state = State(game.get_player_position(), game.get_box_locations(), node.get_path() + action)
                if new_state not in explored:
                    frontier.append(new_state)
                    explored.add(new_state)
                
                game.set_box_positions(node.get_boxes_loc())
                game.set_player_position(node.get_player_loc())
                
    return None, len(explored)

### IDS

In [80]:
# TODO: Must return moves, number of visited states
def solver_ids(game_map):
    game = Game(game_map)
    initial_state = State(game.get_player_position(), game.get_box_locations(), "")
    explored_count = 0
    search_depth = 0
    
    while True:
        has_reached_depth = False
        frontier = [initial_state]
        explored = {}
        
        while frontier:
            node = frontier.pop()
            explored[node] = len(node.get_path())
            explored_count += 1
            game.set_box_positions(node.get_boxes_loc())
            game.set_player_position(node.get_player_loc())
            
            if game.is_game_won():
                return node.get_path(), explored_count
            
            if len(node.get_path()) < search_depth:
                for action in ["U", "D", "L", "R"]:
                    if game.apply_move(action):
                        new_state = State(game.get_player_position(), game.get_box_locations(), node.get_path() + action)
                        if new_state not in explored or explored[new_state] > len(new_state.get_path()):
                            frontier.append(new_state)
                            explored[new_state] = len(new_state.get_path())
                            
                        game.set_box_positions(node.get_boxes_loc())
                        game.set_player_position(node.get_player_loc())
            else:
                has_reached_depth = True
                
        if not has_reached_depth:
            return None, explored_count
        search_depth += 1
        

### A*

In [81]:
def manhattan_dist(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def cal_portal_path(src, dest, portals, val):
    for portal in portals:
        val = min(val, manhattan_dist(src, portal[0]) + manhattan_dist(portal[1], dest), 
                          manhattan_dist(src, portal[1]) + manhattan_dist(portal[0], dest))
    
    if len(portals) > 1:
        val = min(val, 
        manhattan_dist(src, portals[0][0]) + manhattan_dist(portals[0][1], portals[1][0]) + manhattan_dist(portals[1][1], dest), 
        manhattan_dist(src, portals[0][0]) + manhattan_dist(portals[0][1], portals[1][1]) + manhattan_dist(portals[1][0], dest), 
        manhattan_dist(src, portals[0][1]) + manhattan_dist(portals[0][0], portals[1][0])+ manhattan_dist(portals[1][1], dest), 
        manhattan_dist(src, portals[0][1]) + manhattan_dist(portals[0][0], portals[1][1]) + manhattan_dist(portals[1][0], dest))
    
    return val

    
def heuristic(game:Game, naive=False):
    player_loc = game.get_player_position()
    boxes_loc = game.get_box_locations()
    portals_loc = game.get_portal_locations()
    goals_loc = game.get_goal_locations()
    box_to_goals = [0] * len(boxes_loc)
    player_to_boxes = [0] * len(boxes_loc)
    

    for i, box in enumerate(boxes_loc):
        box_to_goal = manhattan_dist(boxes_loc[i], goals_loc[i])
        player_to_box = manhattan_dist(player_loc, box)
        
        if not naive:
            box_to_goals[i] = cal_portal_path(box, goals_loc[i], portals_loc, box_to_goal)
            player_to_boxes[i] = cal_portal_path(player_loc, box, portals_loc, player_to_box)
    
    return sum(box_to_goals) + min(player_to_boxes)


# TODO: Must return moves, number of visited states
def solver_astar(game_map, heuristic_func=heuristic, weight=1):
    game = Game(game_map)
    initial_state = State(game.get_player_position(), game.get_box_locations(), "")
    minheap = []
    heapq.heappush(minheap, (heuristic_func(game), initial_state))
    explored = set()
    path_costs = {}
    path_costs[initial_state] = 0
    
    while minheap:
        node = heapq.heappop(minheap)[1]
        
        game.set_box_positions(node.get_boxes_loc())
        game.set_player_position(node.get_player_loc())
        
        for action in ["U", "D", "L", "R"]:
            if game.apply_move(action):
                new_state = State(game.get_player_position(), game.get_box_locations(), node.get_path() + action)
                if new_state not in explored or (path_costs[node] + 1 < path_costs[new_state]):
                    path_costs[new_state] = path_costs[node] + 1
                    heapq.heappush(minheap, (path_costs[new_state] + weight * heuristic_func(game), new_state))
                    explored.add(new_state)                    
                    if (game.is_game_won()):
                        return new_state.get_path(), len(explored)
                
                game.set_box_positions(node.get_boxes_loc())
                game.set_player_position(node.get_player_loc())
                
    return None, len(explored)


## Solve

In [82]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar
}

In [83]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

In [84]:
solve(10, "A*")

A* took 3.0 seconds on map 10 and visited 58392 states.
46 moves were used: RRRRRDRULURULLLULLDRUULDRDLDRRDRULURURDDRDLLLL


In [85]:
def solve_all():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        # for method in SOLVERS.keys():
        #     if map != 9 and method != "DFS" and method != "IDS":
        #         solve(map, method)
        #     if map != 9 and map != 10 and method == "IDS":
        #         solve(map, "IDS")
        solve(map, "A*")
            

In [86]:
# solve_all() # Solve all maps using all methods